本人是Python&PyTorch新手，为了完成毕业设计而选择学习PyTorch。
在学习过程中，将自己不懂的代码全部进行了注释。
学完了「60分钟入门深度学习工具-PyTorch」这个教程后，觉得自己的注释也许会帮助与我水平差不多的初学者们节省不少时间，所以就把注释详尽的「3.神经网络」、「4.训练一个分类器」分享出来。
注释很多但也显得很乱，建议和原版教程一起看，原版教程看不懂就看一下我的这个版本。

（我其实学了两遍，因为第一遍跟着打了一遍代码，模模糊糊，不知道为什么要这么做，第二遍把几乎每一行代码都进行注释，才感觉自己真正学到了东西）

In [1]:
import torch  # 引入torch模块
import torch.nn as nn  # 引入神经网络模块
import torch.nn.functional as F  # 引入神经网络函数模块


class Net(nn.Module):  # 继承nn模块的Module类（所有网络的基类）
    def __init__(self):
        super(Net, self).__init__()  # 将构造方法委托给Net的父类Module
        self.conv1 = nn.Conv2d(1, 6, 5)  # 第一层二维卷积层，输入1个信号通道，输出6个信号通道，使用5*5的卷积核
        # 第二层二维卷积层，输入6个信号通道，输出16个信号通道，使用5*5的卷积核
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 第一层全连接层，对输入数据线进行线性变换，输入16*5*5个样本，输出120个样本
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)  # 第二层全连接层，对输入数据线进行线性变换，输入120个样本，输出84个样本
        self.fc3 = nn.Linear(84, 10)  # 第三层全连接层，对输入数据线进行线性变换，输入84个样本，输出10个样本

    def forward(self, x):  # 定义每次执行的计算步骤，所有子类都要重写这个函数。
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 将数据集传入，生成第一层卷积层。
        # 使用非线性激活函数relu()处理输入，得出输出（不同分类的概率）
        # 对输入平面conv1进行最大池化，池化窗口大小为2*2
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 将数据集传入，生成第二层卷积层。
        # 使用非线性激活函数relu()处理输入，得出输出（不同分类的概率）
        # 对输入平面conv2进行最大池化，池化窗口大小为2*2（正方形的池化窗口可以直接用单个数字表示）
        x = x.view(-1, self.num_flat_features(x))
        # 将第二次池化后得出的数据集展开成列向量，方便后续的线性运算。
        x = F.relu(self.fc1(x))
        # 将数据集传输fc1层进行线性运算
        # 将线性运算后的结果使用非线性激活函数relu()处理 ，输出
        x = F.relu(self.fc2(x))
        # 将数据集传输fc2层进行线性运算
        # 将线性运算后的结果使用非线性激活函数relu()处理 ，输出
        x = self.fc3(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        # 返回tensor对象的大小(即描述了tensor对象的维度的一个行向量)
        # 然后使用[1:]，将第一个维度，即批处理数量，排除掉。（这里的批处理数量是我自己理解的，原文是the batch dimension，我不确定到底对应 batch 相关的哪个概念）
        num_features = 1  # 接下来是想要计算数据集除了第一维以外的维度的所有元素（特征、feature）的总数量，创建并赋值为1，然后与数据集的其他维度的特征数量依次相乘。
        for s in size:
            num_features *= s
        # 经过上述操作， 就得出了数据集除了第一维以外的维度的所有元素（特征、feature）的总数量，方便之后展平数据集
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
# parameters(memo=None): 返回一个 包含模型所有参数 的迭代器。一般用作optimizer参数
print(len(params))  # params是网络的所有参数的结构、数据，这里len(params)==0，表示网络总共有10层。
for i in range(len(params)):
    print(i, ":", params[i].size())
"""
网络结构为：
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
参数结构为：（箭头后的解释是我自己推测的）
0 : torch.Size([6, 1, 5, 5]) → 第一层卷积层结构：6个输出、1个输入，5*5的卷积核
1 : torch.Size([6]) → 第一个池化层，将第一层卷积层池化后的6个输出
2 : torch.Size([16, 6, 5, 5]) → 第二层卷积层结构：16个输出、6个输入，5*5的卷积核
3 : torch.Size([16]) → 第二个池化层，将第二层卷积层池化后的16个输出
4 : torch.Size([120, 400]) → 第一个全连接层，120个输出，400个输入（将池化层展平为16*5*5）
5 : torch.Size([120]) → 第一个全连接层线性运算后的120个输出作为输入，使用relu()函数运算过后，输出120个参数
6 : torch.Size([84, 120]) → 第二个全连接层，84个输出，120个输入
7 : torch.Size([84]) → 第二个全连接层线性运算后的84个输出作为输入，使用relu()函数运算过后，输出84个参数
8 : torch.Size([10, 84]) → 第三个（最后一个）全连接层，LeNet-5网络模型上写的是Gaussian connections,10个输出（分类），84个输入
9 : torch.Size([10]) → 这里并没有relu()函数的操作，为什么单独出现10个参数作为1层，我不明白，可能需要看一下nn.Module的文档。



"""

10
0 : torch.Size([6, 1, 5, 5])
1 : torch.Size([6])
2 : torch.Size([16, 6, 5, 5])
3 : torch.Size([16])
4 : torch.Size([120, 400])
5 : torch.Size([120])
6 : torch.Size([84, 120])
7 : torch.Size([84])
8 : torch.Size([10, 84])
9 : torch.Size([10])


'\n网络结构为：\nNet(\n  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))\n  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))\n  (fc1): Linear(in_features=400, out_features=120, bias=True)\n  (fc2): Linear(in_features=120, out_features=84, bias=True)\n  (fc3): Linear(in_features=84, out_features=10, bias=True)\n)\n参数结构为：（箭头后的解释是我自己推测的）\n0 : torch.Size([6, 1, 5, 5]) → 第一层卷积层结构：6个输出、1个输入，5*5的卷积核\n1 : torch.Size([6]) → 第一个池化层，将第一层卷积层池化后的6个输出\n2 : torch.Size([16, 6, 5, 5]) → 第二层卷积层结构：16个输出、6个输入，5*5的卷积核\n3 : torch.Size([16]) → 第二个池化层，将第二层卷积层池化后的16个输出\n4 : torch.Size([120, 400]) → 第一个全连接层，120个输出，400个输入（将池化层展平为16*5*5）\n5 : torch.Size([120]) → 第一个全连接层线性运算后的120个输出作为输入，使用relu()函数运算过后，输出120个参数\n6 : torch.Size([84, 120]) → 第二个全连接层，84个输出，120个输入\n7 : torch.Size([84]) → 第二个全连接层线性运算后的84个输出作为输入，使用relu()函数运算过后，输出84个参数\n8 : torch.Size([10, 84]) → 第三个（最后一个）全连接层，LeNet-5网络模型上写的是Gaussian connections,10个输出（分类），84个输入\n9 : torch.Size([10]) → 这里并没有relu()函数的操作，为什么单独出现10个参数作为1层，我不明白，可能需要看一下nn.Module的文档。

In [3]:
input = torch.randn(1, 1, 32, 32)  # 创建随机输入数据集，尺寸是LeNet-5网络模型期望的32*32
# 但是这个 1,1 是干嘛的？


out = net(input)  # 将输入数据集传入神经网络net，返回预测结果。
print(out)

tensor([[-0.0031,  0.0207, -0.0010, -0.0729,  0.1585, -0.0972, -0.0552, -0.0612,
         -0.0253,  0.0905]], grad_fn=<AddmmBackward>)


In [5]:
print(net.conv1.bias.grad)
net.zero_grad()  # 将所有参数的梯度缓存清零，可以本来就没有缓存啊？
print(net.conv1.bias.grad)
out.backward(torch.randn(1, 10))  # 然后进行随机梯度的反向传播。
# 梯度值是反向传播之后才有的。那.zero_grad()可能是教程中为了保险起见，先置零之前计算的梯度吧。
print(net.conv1.bias.grad)

None
None
tensor([ 0.0124, -0.0275, -0.0780, -0.1368,  0.0101, -0.0190])


In [4]:
# 接下来是计算损失函数

output = net(input)  # 将输入数据集传入神经网络net，返回预测结果。
target = torch.randn(10)  # 创建一个数据集，作为期望预测出的目标数据
print(target)

tensor([ 1.1139, -0.7768,  0.0033, -1.7236,  0.6454,  0.7855, -0.6819, -2.6723,
         0.4511,  0.1755])


In [6]:
# 将目标数据集展平（这里创建时就是一个可以直接用的行向量，但是实际项目中可能是有很多维度），和输出数据集维度相同
target = target.view(1, -1)
print(target)

tensor([[ 1.1139, -0.7768,  0.0033, -1.7236,  0.6454,  0.7855, -0.6819, -2.6723,
          0.4511,  0.1755]])


In [7]:
criterion = nn.MSELoss()
"""
我其实不理解把一个函数赋值给变量是什么操作？
哦，应该是这个函数的返回值复制给了criterion。
但是为什么打印criterion就是这个函数啊？
"""
print(criterion)

MSELoss()


In [8]:
print(net.conv1.bias.grad)

tensor([ 0.0124, -0.0275, -0.0780, -0.1368,  0.0101, -0.0190])


In [9]:
loss = criterion(output, target)  # 计算损失函数
print(loss)

tensor(1.3070, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

"""
这是教程中说的话，最后一句我没太明白：

反向跟踪loss,使用它的.grad_fn属性,会看到向下面这样的一个计算图:
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss

所以, 当你调用loss.backward(),整个图被区分为损失以及图中所有具有requires_grad = True的张量，并且其.grad 张量的梯度累积。
"""

'\n这是教程中说的话，最后一句我没太明白：\n\n反向跟踪loss,使用它的.grad_fn属性,会看到向下面这样的一个计算图:\ninput -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss\n\n所以, 当你调用loss.backward(),整个图被区分为损失以及图中所有具有requires_grad = True的张量，并且其.grad 张量的梯度累积。\n'

In [11]:
print(net.conv1.bias.grad)
net.zero_grad()  # 将梯度置零
print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

tensor([ 0.0124, -0.0275, -0.0780, -0.1368,  0.0101, -0.0190])
conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [12]:
loss.backward()  # 反向传播，观察梯度变化。
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0145, -0.0062, -0.0054,  0.0168,  0.0050,  0.0067])


In [13]:
"""
这一块内容是用Python基础语法来实现更新权重，而PyTorch封装好了各种函数（如torch.optim.SGD()，可以直接调用
"""
learning_rate = 0.01
for f in net.parameters():
    # 使用 .data 返回和tensor对象的数据 , 但不会加入到tensor对象的计算历史里，且require s_grad = False
    f.data.sub_(f.grad.data * learning_rate)
    #

In [14]:
import torch.optim as optim
# torch.optim是一个实现了各种优化算法的库。大部分常用的方法得到支持，并且接口具备足够的通用性，使得未来能够集成更加复杂的方法。
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 为了构建一个Optimizer，你需要给它一个包含了需要优化的参数（必须都是Variable对象）的iterable。然后，你可以设置optimizer的参 数选项，比如学习率，权重衰减，等等。
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [15]:
optimizer.zero_grad()  # 还是没懂为什么要置零梯度，可能是optimizer作为结果参数，和之前的一些结果参数有同样的创建参数，之前的创建参数有不
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [16]:
output = net(input)
print(output)

tensor([[ 0.0124,  0.0106, -0.0066, -0.0828,  0.1654, -0.0868, -0.0538, -0.0921,
         -0.0187,  0.0879]], grad_fn=<AddmmBackward>)


In [17]:
loss = criterion(output, target)
loss.backward()
optimizer.step()
# 反向传播之后，得出不同输入的梯度，然后以此更新不同输入的权重。